In [ ]:
# SPDX-FileCopyrightText: Copyright (c) 2024 NVIDIA CORPORATION & AFFILIATES. All rights reserved.
# SPDX-License-Identifier: LicenseRef-NvidiaProprietary
#
# NVIDIA CORPORATION, its affiliates and licensors retain all intellectual
# property and proprietary rights in and to this material, related
# documentation and any modifications thereto. Any use, reproduction,
# disclosure or distribution of this material and related documentation
# without an express license agreement from NVIDIA CORPORATION or
# its affiliates is strictly prohibited.

# Accelerating Quantum Computing: A Step-by-Step Guide to Expanding SimulationCapabilities and Enabling Interoperability of Quantum Hardware
  
## Overview of methods of accelerating quantum simulation with GPUs.

* Introduction to CUDA-Q through three Hello World examples using `sample` and `observe` calls.  
* Guide to different simulation backends for executing quantum circuits, emphasizing a variety of patterns of parallelization: 
    * statevector memory over multiple processors
    * circuit sampling over multiple processors
    * Hamiltonian batching
    * circuit cutting
* Live-demo installation and demonstration of how to access the NV Quantum Cloud.


### Hello World Examples 

In [2]:
import cudaq
from cudaq import spin
from typing import List
import numpy as np

In [5]:
# Example 1 - sampling a circuit

##############################################################
#  1. Select a backend for kernel execution
cudaq.set_target("qpp-cpu")
##############################################################

##############################################################
# 2. Define a kernel function 
@cudaq.kernel
def kernel(qubit_count: int):
    # Allocate our `qubit_count` to the kernel.
    qvector = cudaq.qvector(qubit_count)

    # Apply a Hadamard gate to the qubit indexed by 0.
    h(qvector[0])
    # Apply a Controlled-X gate between qubit 0 (acting as the control)
    # and each of the remaining qubits.  
    for i in range(1, qubit_count):
        x.ctrl(qvector[0], qvector[i])

    # Measure the qubits
    # If we don't specify measurements, all qubits are measured in
    # the Z-basis by default.
    mz(qvector)

##############################################################
# 3. Call the kernel function with the variable qubit_count set to 2 and sample the outcomes
qubit_count = 2
result = cudaq.sample(kernel, qubit_count, shots_count=1000)

print(result)

{ 00:534 11:466 }



In [6]:
# Example 2 - Expectation value calculations

# Define a quantum kernel function.
@cudaq.kernel
def kernel(qubit_count: int):
    # Allocate our `qubit_count` to the kernel.
    qvector = cudaq.qvector(qubit_count)

    # Apply a Hadamard gate to the qubit indexed by 0.
    h(qvector[0])
    # Apply a Controlled-X gate between qubit 0 (acting as the control)
    # and each of the remaining qubits.  
    for i in range(1, qubit_count):
        x.ctrl(qvector[0], qvector[i])

# Define a Hamiltonian in terms of Pauli Spin operators.
hamiltonian = spin.z(0) + 2*spin.y(1) - spin.x(0) * spin.z(1)

# Compute the expectation value given the state prepared by the kernel.
result = cudaq.observe(kernel, hamiltonian, qubit_count, shots_count = 1000).expectation()

print('<psi|H|psi> =', result)

<psi|H|psi> = -0.030000000000000027


### Guide to Different Simulation Targets


The figure below illustrates a few options for accelerating statevector simulations of single quantum processor kernel executions: one CPU, one GPU, or a multi-node, multi-GPU system. 

![](images/single-processor-backends.jpg)

In the Hello World examples in the previous section, we saw statevector simulations of a QPU on a CPU.  When GPU resources are available, we can use a single-GPU or multi-node, multi-GPU systems for fast statevector simulations. The `nvidia` target accelerates statevector simulations through `cuStateVec` library. This target offers a variety of configuration options:

* **Single-precision GPU simulation** (default): Setting the target to `nvidia` through command `cudaq.set_target('nvidia')` provides single (`fp32`) precision statevector simulation on one GPU as the default.

* **Double fp64 precision on a single-GPU**: The option `cudaq.set_target('nvidia', option='fp64')` increases the precision of the statevector simulation on one GPU.

* **Multi-node, multi-GPU simulation**: To run the `cuStateVec` simulator on multiple GPUs, set the target to `nvidia` with the `mgpu` option (`cudaq.set_target('nvidia', option='mgpu,fp64')`) and then run the python file containing your quantum kernels within a `MPI` context: `mpiexec -np 2 python3 program.py`. Adjust the `-np` tag according to the number of GPUs you have available.




Next, we'll cover a few of the ways you can organize the distribution of quantum simulations over multiple GPU processors, whether you are simulating a single QPU or multiple QPUs.

#### Single QPU Statevector Simulations

 


In some cases, the memory required to hold the entire statevector exceeds the memory of a single GPU. In these cases we can distribute the statevector across multiple GPUs as the diagram in the image below suggests.  

![](images/statevector-distribution.png)

This is handled automatically within the `mgpu` option when the number of qubits in the statevector exceeds 25.  By changing the environmental variable `CUDAQ_MGPU_NQUBITS_THRESH` prior to setting the target, you can change the threshold in which statevector distribution is invoked.








#### Simulating Parallel QPU computaiton

There are several patterns for multi-QPU computation. We'll examine a few of them here:

* Circuit sampling distributed over multiple processors
* Hamiltonian batching
* Circuit cutting


##### Circuit Sampling

One method of parallelization is to sample a circuit over several processors as illustrated in the diagram below.

![](images/circuit-sampling.png)

The following code illustrates how to launch asynchronous sampling tasks using `sample_async` on multiple virtual QPUs, each simulated by a tensornet simulator backend using the `remote-mqpu` target.

In [ ]:
# Specified as program input, e.g.
# ```
backend = "tensornet"; servers = "2"
# ```
#backend = args.backend
#servers = args.servers

# Define a kernel to be sampled.
@cudaq.kernel
def kernel(controls_count: int):
    controls = cudaq.qvector(controls_count)
    targets = cudaq.qvector(2)
    # Place controls in superposition state.
    h(controls)
    for target in range(2):
        x.ctrl(controls, targets[target])
    # Measure.
    mz(controls)
    mz(targets)

# Set the target to execute on and query the number of QPUs in the system;
# The number of QPUs is equal to the number of (auto-)launched server instances.
cudaq.set_target("remote-mqpu",
                backend=backend,
                auto_launch=str(servers) if servers.isdigit() else "",
                url="" if servers.isdigit() else servers)
qpu_count = cudaq.get_target().num_qpus()
print("Number of virtual QPUs:", qpu_count)

# We will launch asynchronous sampling tasks,
# and will store the results as a future we can query at some later point.
# Each QPU (indexed by an unique Id) is associated with a remote REST server.
count_futures = []
for i in range(qpu_count):

    result = cudaq.sample_async(kernel, i + 1, qpu_id=i)
    count_futures.append(result)
print("Sampling jobs launched for asynchronous processing.")

# Go do other work, asynchronous execution of sample tasks on-going.
# Get the results, note future::get() will kick off a wait
# if the results are not yet available.
for idx in range(len(count_futures)):
    counts = count_futures[idx].get()
    print(counts)

##### Hamiltonian Batching
Another option for distributing the computation required in a simulation is through Hamiltonian batching in which expectation values of terms of the Hamiltonian are computed in parallel across multiple virtual QPUs as in the image below.

![](images/Hamiltonian-batching.png)

To distribute the expectation value computations of a multi-term Hamiltonian across multiple virtual QPUs use the `nvidia-mqpu` platform as in the following example:


In [ ]:
cudaq.set_target("nvidia", option="mqpu")
target = cudaq.get_target()
num_qpus = target.num_qpus()
print("Number of QPUs:", num_qpus)


# Define spin ansatz.
@cudaq.kernel
def kernel(angle: float):
    qvector = cudaq.qvector(2)
    x(qvector[0])
    ry(angle, qvector[1])
    x.ctrl(qvector[1], qvector[0])


# Define spin Hamiltonian.
hamiltonian = 5.907 - 2.1433 * spin.x(0) * spin.x(1) - 2.1433 * spin.y(
    0) * spin.y(1) + .21829 * spin.z(0) - 6.125 * spin.z(1)

exp_val = cudaq.observe(kernel,
                        hamiltonian,
                        0.59,
                        execution=cudaq.parallel.thread).expectation()
print("Expectation value: ", exp_val)

In the above code snippet, since the Hamiltonian contains four non-identity terms, there are four quantum circuits that need to be executed in order to compute the expectation value of that Hamiltonian and given the quantum state prepared by the ansatz kernel. When the nvidia-mqpu platform is selected, these circuits will be distributed across all available QPUs. The final expectation value result is computed from all QPU execution results.  

Another option for Hamiltonian batching is to use the MPI context and multiple GPUs.  You can read more about this [here](https://nvidia.github.io/cuda-quantum/latest/using/backends/platform.html#nvidia-mqpu-platform).

##### Circuit cutting

Circuit cutting is a common pattern for parallelization. One way of visualizing circuit cutting is through the max cut problem. In this example, we aim to approximate the max cut of a graph with the divide-and-conquer also referred to as QAOA-in-QAOA or QAOA^2 approach which breaks the graph down into smaller subgraphs and solves the max cuts for these subgraphs in parallel using QAOA (see for instance [arXiv:2205.11762v1](https://arxiv.org/abs/2205.11762), [arxiv.2101.07813v1](https://arxiv.org/abs/2101.07813), [arxiv:2304.03037v1](https://arxiv.org/abs/2304.03037), [arxiv:2009.06726](https://arxiv.org/abs/2009.06726), and [arxiv:2406:17383](https://arxiv.org/abs/2406.17383)).  By doing this we have effectively cut the QAOA circuit for the larger graph into the smaller QAOA for circuits for the subgraphs.  To complete the circuit cutting, we'll need to merge the results of QAOA on the subgraphs into a result for the entire graph.  This requires solving another smaller optimization problem, which can also be tackled with QAOA.  You can read it about that in more detail in a series of [interactive labs](https://github.com/NVIDIA/cuda-q-academic/tree/main/qaoa-for-max-cut).

![](images/circuit-cutting.png)

This example illustrates how to use the `MPI` context to orchestrate running `@cudaq.kernel` decorated functions in parallel. Additionally, a few exercises are built into this longer example to gain some practice with the CUDA-Q commands introduced earlier in this notebook. Solutions to these exercises appear in the solutions.ipynb file, but we encourage you to first attempt the exercises out yourself.

First we need to define a graph and subgraphs.  Execute the cell below to generate the graph and subgraphs for the divide-and-conquer QAOA. For this demonstrate, we'll stick with a small graph and divide it into five smaller graphs.

In [20]:
# Define an example graph
edgeList = [(0,1), (1,2), (2,3), (3,0), (0,2), (2,4), (3,4), (4,5), (3,5)]

# Identify subgraphs, separating out the edges as source and target nodes
nodeCountList = [8,7,6,5,4]
nodeList : List[int] = []
edgeListSources : List[int] = []
edgeListTargets : List[int] = []

# subgraph0 data
nodeList.append([3, 6, 9, 10, 13, 14, 21, 22])
edgeListSources.append([3,3,3,3,6,6,9,14])
edgeListTargets.append([14,9,10,13,22,13,21,22])

# subgraph1 data
nodeList.append([8, 11, 12, 15, 16, 25, 26])
edgeListSources.append([8, 8, 11, 11, 11, 11, 12, 15, 16, 16, 25])
edgeListTargets.append([25, 12, 26, 25, 15, 12, 15, 16, 25, 26, 26])

Next let's create kernels for the QAOA circuit.  

In [21]:
# Problem Kernel

@cudaq.kernel
def qaoaProblem(qubit_0 : cudaq.qubit, qubit_1 : cudaq.qubit, alpha : float):
    """Build the QAOA gate sequence between two qubits that represent an edge of the graph
    Parameters
    ----------
    qubit_0: cudaq.qubit
        Qubit representing the first vertex of an edge
    qubit_1: cudaq.qubit
        Qubit representing the second vertex of an edge
    alpha: float
        Free variable

    """
    x.ctrl(qubit_0, qubit_1)
    rz(2.0*alpha, qubit_1)
    x.ctrl(qubit_0, qubit_1)

# Mixer Kernel
@cudaq.kernel
def qaoaMixer(qubit_0 : cudaq.qubit, beta : float):
    """Build the QAOA gate sequence that is applied to each qubit in the mixer portion of the circuit
    Parameters
    ----------
    qubit_0: cudaq.qubit
        Qubit
    beta: float
        Free variable

    """
    rx(2.0*beta, qubit_0)


# We now define the kernel_qaoa function which will be the QAOA circuit for our graph
@cudaq.kernel
def kernel_qaoa(qubit_count :int, layer_count: int, qubits_src: List[int], qubits_tgt: List[int], thetas : List[float]):
    """Build the QAOA circuit for max cut of the graph with given edges and nodes
    Parameters
    ----------
    qubit_count: int
        Number of qubits in the circuit, which is the same as the number of nodes in our graph
    layer_count : int
        Number of layers in the QAOA kernel
    edges_src: List[int]
        List of the first (source) node listed in each edge of the graph, when the edges of the graph are listed as pairs of nodes
    edges_tgt: List[int]
        List of the second (target) node listed in each edge of the graph, when the edges of the graph are listed as pairs of nodes
    thetas: List[float]
        Free variables to be optimized
    """
    # Let's allocate the qubits
    qreg = cudaq.qvector(qubit_count)

    # And then place the qubits in superposition
    h(qreg)
    
    # Each layer has two components: the problem kernel and the mixer
    for i in range(layer_count):
        # Add the problem kernel to each layer
        for edge in range(len(qubits_src)):
            qubitu = qubits_src[edge]
            qubitv = qubits_tgt[edge]
            qaoaProblem(qreg[qubitu], qreg[qubitv], thetas[i])
        # Add the mixer kernel to each layer
        for j in range(qubit_count):
            qaoaMixer(qreg[j],thetas[i+layer_count])

In [22]:
# We'll create this function to rename the nodes to be sequential integers
# beginning with 0 as a way to map the graph nodes to qubits. This is
# necessary because we can't use the .index option for lists
# within a cudaq.kernel
def rename_nodes(edge_src, edge_tgt, nodes):
    """ 
    Parameters
    ----------
    edges_src: List[int]
        List of the first (source) node listed in each edge of the graph, when the edges of the graph are listed as pairs of nodes
    edges_tgt: List[int]
        List of the second (target) node listed in each edge of the graph, when the edges of the graph are listed as pairs of nodes
    nodes: List[int]
        List of nodes of the graph
    
    Returns
    -------
    new_edge_src : List[int]
        List of the first (source) node listed in each edge of the graph after renaming nodes to be sequential integers beginning with 0, 
        when the edges of the graph are listed as pairs of nodes
    new_edge_tgt : List[int]
        List of the second (target) node listed in each edge of the graph after renaming nodes to be sequential integers beginning with 0, 
        when the edges of the graph are listed as pairs of nodes
    """
    new_edge_src = []
    new_edge_tgt = []
    for i in range(len(edge_src)):
        new_edge_src.append(nodes.index(edge_src[i]))
        new_edge_tgt.append(nodes.index(edge_tgt[i]))
    return new_edge_src, new_edge_tgt

    

In [23]:
new_src, new_tgt= rename_nodes([3,3,3,3,6,6,9,14],[14,9,10,13,22,13,21,22],[3, 6, 9, 10, 13, 14, 21, 22])
#cudaq.draw(kernel_qaoa,8, 1,new_src, new_tgt, [0.3811067525772551, 2.752504235882645])
results = cudaq.sample(kernel_qaoa,8, 1,new_src, new_tgt, [0.3811067525772551, 2.752504235882645], shots_count=1000)
print(results)

{ 11111100:4 11110101:1 11110100:2 11110011:2 11110001:3 01110111:2 01110110:12 01110100:31 01101010:2 01101001:1 01100110:3 00101011:3 01100101:1 01100100:12 01101100:12 10111100:1 01011110:24 01010011:2 11110000:9 10000001:13 01001011:2 01110011:2 10010001:5 11010011:9 01000110:2 01000011:1 01001110:5 11000001:11 11000000:2 00110101:3 00011100:1 00111011:14 11111110:1 00111001:29 10000110:3 10001110:11 10100011:2 00101110:3 01011010:4 00011110:16 10001100:7 10011010:2 01000000:3 00000001:1 01111110:7 10011111:1 01000010:2 10001011:50 01110101:6 00001100:1 00000011:3 00011111:26 01110001:12 01010110:16 11001100:2 00100101:2 00101001:18 11111001:2 01101110:3 00001111:1 00111111:8 10001000:3 11010110:13 00101100:11 00010010:1 10011011:11 00001110:9 01100011:3 01010111:3 01000111:1 00101101:12 00011011:24 10111011:1 00111101:29 10101000:1 00111100:23 10111110:2 01010100:1 00111110:5 00011001:1 01111100:35 01111111:1 10000011:31 10000100:1 10100110:1 00000111:1 10000101:1 01111000:4 10101

We'll need a Hamiltonian to encode the cost function.  

In [29]:
# Define a function to generate the Hamiltonian for a max cut problem using the graph G

def hamiltonian_max_cut(sources : List[int], targets : List[int]):
    """Hamiltonian for finding the max cut for the graph  with edges defined by the pairs generated by source and target edges

    Parameters
    ----------
    sources: List[int]
        list of the source vertices for edges in the graph
    targets: List[int]
        list of the target vertices for the edges in the graph

    Returns
    -------
    cudaq.SpinOperator
        Hamiltonian for finding the max cut of the graph defined by the given edges
    """
    hamiltonian = 0
   
    # Since our vertices may not be a list from 0 to n, or may not even be integers,
    # we need to map the vertices to the list of integers 0 to qubit_count -1
    
    for i in range(len(sources)):
        # Add a term to the Hamiltonian for the edge (u,v)
        qubitu = sources[i]
        qubitv = targets[i]
        hamiltonian += 0.5*(spin.z(qubitu)*spin.z(qubitv)-spin.i(qubitu)*spin.i(qubitv))

    return hamiltonian

Now let's put this all together in a function that finds the the optimal parameters for QAOA of a given subgraph.

In [30]:
def find_optimal_parameters(qubit_src : List[int], qubit_tgt : List[int], qubit_count : int, layer_count: int, seed :int):
    """Function for finding the optimal parameters of QAOA for the max cut of a graph
    Parameters
    ----------
    qubit_src: List[int]
    qubit_tgt: List[int]
        Sources and targets defining the edges of the graph
    nodes: List[int]
        Integer labels of the nodes of the graph
    qubit_count: int
        qubit_count is the number of nodes in the graph
    layer_count : int
        Number of layers in the QAOA circuit
    seed : int
        Random seed for reproducibility of results

    Returns
    -------
    list[float]
        Optimal parameters for the QAOA applied to the given graph
    """
    
    # Each layer of the QAOA kernel contains 2 parameters
    parameter_count : int = 2*layer_count

    # Specify the optimizer and its initial parameters.
    optimizer = cudaq.optimizers.COBYLA()
    np.random.seed(seed)
    optimizer.initial_parameters = np.random.uniform(-np.pi, np.pi,
                                                     parameter_count)
    
    # Pass the kernel, spin operator, and optimizer to `cudaq.vqe`.
    optimal_expectation, optimal_parameters = cudaq.vqe(
        kernel=kernel_qaoa,
        spin_operator=hamiltonian_max_cut(qubit_src, qubit_tgt),
        argument_mapper=lambda parameter_vector: (qubit_count, layer_count, qubit_src, qubit_tgt, parameter_vector),
        optimizer=optimizer,
        parameter_count=parameter_count)

    return optimal_parameters

Before running this function in parallel, let's execute it sequentially. 

In [ ]:
# Testing the find_optimal_parameters function
layer_count = 1
seed = 123


for i in [0,1]:
    new_src, new_tgt= rename_nodes(edgeListSources[i], edgeListTargets[i], nodeList[i])
    optimal_parameters = find_optimal_parameters(new_src, new_tgt, nodeCountList[i], layer_count, seed)
    print(optimal_parameters)

[2.760265752934724, -1.1781480121902836]
[2.863390666464872, -1.2262588372574177]


Finally, we'll need to sample the `kernel_qaoa` circuit with the optimal parameters to find approximate max cut solutions to each of the subgraphs.

To learn more about how the results of the subgraph solutions are merged together to get a max cut approximation of the original graph, check out the 2nd notebook of this [series of interactive tutorials](https://github.com/NVIDIA/cuda-q-academic/tree/main/qaoa-for-max-cut).

In [34]:
shots = 10000
optimal_parameters = [2.760265752934724, -1.1781480121902836]
layer_count = 1

for i in [0,1]:
    new_src, new_tgt= rename_nodes(edgeListSources[i], edgeListTargets[i], nodeList[i])
    counts = cudaq.sample(kernel_qaoa, nodeCountList[i], layer_count, new_src, new_tgt, optimal_parameters, shots_count=shots)
    print('most_probable outcome = ',counts.most_probable())
    results = str(counts.most_probable())

most_probable outcome =  10001011
most_probable outcome =  1100100


To execute the workflow below on 4 processors (these can be separate GPUs or separate processes on a single GPU), we'll use MPI and the `rank` variable.

![](images/parallel-workflow.png)

### Beyond Statevector Simulations

#### Other simulators

If an NVIDIA GPU and CUDA runtime libraries are available, the default target is set to `nvidia`. This will utilize the cuQuantum single-GPU state vector simulator. On CPU-only systems, the default target is set to `qpp-cpu` which uses the OpenMP CPU-only simulator.

For many applications, it's not necessary to simluate and access the entire statevector. The default simulator can be overridden by the environment variable CUDAQ_DEFAULT_SIMULATOR where tensor network, matrix product state simulators can be selected. Please refer to the table below for a list of backend simulator names along with its multi-GPU capability.

![](images/backends.png)

For more information about all the simulator backends available on [this documentation page](https://nvidia.github.io/cuda-quantum/latest/using/backends/simulators.html#tensor-network-simulators).

#### Quantum processing units
In addition to executing simulations, CUDA-Q is equipped to run quantum kernels on quantum processing units.  For more information on how to execute CUDA-Q code on quantum processing units, check out the [documentation](https://nvidia.github.io/cuda-quantum/latest/using/backends/hardware.html).

## Next up:
[Installing CUDA-Q locally](https://nvidia.github.io/cuda-quantum/latest/using/quick_start.html#install-cuda-q) and [accessing the NVQC](https://nvidia.github.io/cuda-quantum/latest/using/backends/nvqc.html).